# Text Analysis

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional 
from keras.optimizers import RMSprop

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence

from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np

from sklearn.utils import shuffle

import re

import gensim
#from pymystem3 import Mystem
#from stanfordcorenlp import StanfordCoreNLP
import spacy_udpipe


Using TensorFlow backend.


In [2]:
df = pd.read_csv('./task2_data/task2_en_training.tsv',delimiter='\t')
df.head()

,tweet_id,user_id,class,tweet
0,344266386467606528,809439366,0,"depression hurts, cymbalta can help"
1,349220537903489025,323112996,0,"@jessicama20045 right, but cipro can make thin..."
2,351421773079781378,713100330,0,@fibby1123 are you on paxil .. i need help
3,326594278472171520,543113070,0,@redicine the lamotrigine and sjs just made ch...
4,345567138376994816,138795534,0,have decided to skip my #humira shot today. my...


In [3]:
spacy_udpipe.download("en")
nlp = spacy_udpipe.load("en")
model = gensim.models.KeyedVectors.load_word2vec_format("./Embeddings/model.bin", binary=True) 
model.vocab 


Already downloaded a model for the 'en' language


{'say_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443cca90>,
 'go_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccad0>,
 'make_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccb90>,
 'get_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccbd0>,
 'one_NUM': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccc50>,
 'see_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccd10>,
 'time_NOUN': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccdd0>,
 'take_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443cce50>,
 'know_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443cced0>,
 'year_NOUN': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccf10>,
 'two_NUM': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccf50>,
 'well_ADV': <gensim.models.keyedvectors.Vocab at 0x7fa7443ccfd0>,
 'come_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ef0d0>,
 'think_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7443ef150>,
 'give_VERB': <gensim.models.keyedvectors.Vocab at 0x7fa7

In [4]:
def lemmatize_with_postag(text = """"Wikipedia is a free online encyclopedia, created and 
                          edited by volunteers around the world.""",
                          nlp=nlp):
    doc = nlp(text)
    tagged = []
    for token in doc:
        lemma = token.lemma_
        pos = token.pos_
        
        #pos = pos.split('=')[0].strip()
        if pos not in ["PUNCT"]: #["VERB", "NUM", "ADV", "NOUN", "ADJ", "AUX", "PRON", "ADP", "DET", "ADV", "INTJ", "PROPN"]:
            tagged.append(lemma.lower() + '_' + pos)
        #else : print(pos)
    return tagged
   
#print(lemmatize_with_postag())

In [5]:
X_train = df['tweet']
y_train = df['class']

Xy = pd.concat([X_train, y_train], axis=1)
#print(Xy_train)

n_items_for_train = 1500
n_items_for_test = 500

df_1 = Xy[Xy['class'] == 1].iloc[:n_items_for_train] # n_items_for_train твитов для тренировки с индексом класса 1
df_0 = Xy[Xy['class'] == 0].iloc[:n_items_for_train] # n_items_for_train твитов для тренировки с индексом класса 0
#print(df_0.shape)
Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_train = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)
#print(Xy_train['tweet'].apply(lemmatize_with_postag).shape)

df_1 = Xy[Xy['class'] == 1].iloc[n_items_for_train:n_items_for_train+n_items_for_test] # n_items_for_test твитов для теста с индексом класса 1
df_0 = Xy[Xy['class'] == 0].iloc[n_items_for_train:n_items_for_train+n_items_for_test] # n_items_for_test твитов для теста с индексом класса 0
Xy_ = shuffle(pd.concat([df_0, df_1]))
Xy_test = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)

print(Xy_train.shape)
print(Xy_test.shape)
Xy_train, Xy_test

(3000, 2)
(903, 2)


(                                                   tweet  class
 1472   [wake_VERB, up_ADP, from_ADP, a_DET, nap_NOUN,...      1
 520    [@kj_barnard_ADJ, hello_NOUN, i_PRON, take_VER...      0
 2906   [@lithiumlibgirl_VERB, my_PRON, last_ADJ, trip...      1
 1990   [@mereprototype_VERB, probably_ADV, the_DET, c...      1
 16769  [@tonyrobinson666_NUM, @tonyrobinson666_NOUN, ...      1
 ...                                                  ...    ...
 7531   [@msozhild_ADJ, idk_NOUN, of_ADP, any_DET, jus...      1
 1184   [i_PRON, need_VERB, lot_NOUN, and_CCONJ, lot_N...      0
 13305  [why_ADV, must_AUX, lunesto_PART, make_VERB, m...      1
 324    [i_PRON, get_VERB, separation_NOUN, anxiety_NO...      0
 1285   [definition_NOUN, of_ADP, depression_NOUN, you...      0
 
 [3000 rows x 2 columns],
                                                    tweet  class
 20305  [so_ADV, i_PRON, have_AUX, be_AUX, on_ADP, met...      1
 20358  [lowkey_ADJ, high_ADJ, af_NOUN, in_ADP, class_...     

In [6]:
ttws = text_to_word_sequence(X_train[1], filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')
#ttws

In [7]:
max_words = 3000
max_len = 80
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Xy_train["tweet"]) #fit on train

# exctrax features from train
sequences_train = tok.texts_to_sequences(Xy_train["tweet"])
sequences_matrix_train = sequence.pad_sequences(sequences_train, maxlen=max_len)

# exctrax features from test
sequences_test = tok.texts_to_sequences(Xy_test["tweet"])
sequences_matrix_test = sequence.pad_sequences(sequences_test, maxlen=max_len)

In [8]:
def fix_word_key(string):
    upper_part = (re.findall('_.*',string))[0].upper()
    result = re.sub(r'_.*', upper_part, string)
    #print(f'was {string} -> became {result}')
    return result

word_index = tok.word_index
print('Found %s unique tokens' % len(word_index))

# fix tokenizer problem
for key in word_index.keys():
    fixed_key = fix_word_key(key)
    word_index[fixed_key] = word_index.pop(key)
    
print('Found %s unique tokens' % len(word_index))
len(word_index)

Found 9095 unique tokens
Found 9095 unique tokens


9095

In [9]:
# делаем Embedding на основе w2v модели

nb_words = min(max_words, len(word_index))+1 # проверяем где меньше, в нашем датасете или в токенайзере.
EMBEDDING_DIM = 300 # размерность векторов в нашей модели w2v

print(f'number of words = {nb_words}')

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
print(embedding_matrix.shape)

counter = 0
for word, i in word_index.items():
    if word in model.vocab:
        #print(model[word])
        embedding_matrix[i] = model[word]
    counter += 1
    if counter > nb_words: break
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

pd.DataFrame(embedding_matrix)

number of words = 3001
(3001, 300)
Null word embeddings: 2218


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
########################################
## define the model structure
########################################

rate_drop_lstm  = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25
act = 'relu'


embedding_layer      = Embedding(nb_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=max_len,
                                trainable=False) 
lstm_layer           = Bidirectional(LSTM(64, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
sequence_1_input     = Input(shape=(max_len,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1                   = lstm_layer(embedded_sequences_1)
merged               = Dropout(rate_drop_dense)(x1)
merged               = BatchNormalization()(merged)
merged               = Dense(32, activation=act)(merged)
merged               = Dropout(rate_drop_dense)(merged)
merged               = BatchNormalization()(merged)
preds                = Dense(1, activation='sigmoid')(merged)
#preds                = Dense(2, activation='softmax')(merged)

In [11]:
########################################
## train the model
########################################
STAMP = 'lstm_%d_%d_%.2f_%.2f'%(10, 10, rate_drop_lstm, \
        rate_drop_dense)

model = Model(inputs=[sequence_1_input], outputs=preds)
model.compile(loss='binary_crossentropy', 
              optimizer='nadam', 
              metrics=['acc'])
model.summary()
print(STAMP, end='\n\n')

early_stopping =EarlyStopping(monitor='val_loss', patience=30)
bst_model_path = STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit(sequences_matrix_train, Xy_train["class"], 
        validation_data=(sequences_matrix_test[:n_items_for_test//2], 
                         Xy_test["class"][:n_items_for_test//2]), 
        epochs=200, batch_size=100, shuffle=True, \
        callbacks=[early_stopping, model_checkpoint])
    
# to_categorical(Xy_train["class"], num_classes=2, dtype='float32')
# hist = model.fit(sequences_matrix_train, to_categorical(Xy_train["class"], num_classes=2, dtype='float32'), 
#                  validation_data=(sequences_matrix_test, to_categorical(Xy_train["class"], num_classes=2, dtype='float32')), 
#                  epochs=200, batch_size=100, shuffle=True, 
#                  callbacks=[early_stopping, model_checkpoint])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 80)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 80, 300)           900300    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0   

In [12]:
pred = model.predict(sequences_matrix_test[n_items_for_test//2:])
pred.shape
bin_pred, real = [], np.array(Xy_test["class"]).tolist()[n_items_for_test//2:]
for i in range(len(pred)):
    #print(f'pred = {int(round(float(pred[i][0]), 0))},\t real = {real[i]}')
    bin_pred.append(int(round(float(pred[i][0]), 0))) 
#np.array(Xy_test["class"]).tolist()
bin_pred, real
from sklearn.metrics import classification_report
print(classification_report(real, bin_pred))

              precision    recall  f1-score   support

           0       0.68      0.80      0.74       346
           1       0.72      0.58      0.65       307

    accuracy                           0.70       653
   macro avg       0.70      0.69      0.69       653
weighted avg       0.70      0.70      0.69       653



In [13]:
# # exctrax features from test
# Xy = pd.concat([X_train, y_train], axis=1)
# df_1 = Xy[Xy['class'] == 1].iloc[4:4+n_items_for_test] # n_items_for_test твитов для теста с индексом класса 0
# df_0 = Xy[Xy['class'] == 0].iloc[4:4+n_items_for_test] # n_items_for_test твитов для теста с индексом класса 0
# Xy_ = shuffle(pd.concat([df_0, df_1]))
# Xy_test_test = pd.concat([Xy_["tweet"].apply(lemmatize_with_postag), Xy_["class"]], axis = 1)

# # ddf
# sequences_test_test = tok.texts_to_sequences(Xy_test_test["tweet"])
# sequences_matrix_test_test = sequence.pad_sequences(sequences_test_test, maxlen=max_len)

# pred = model.predict(sequences_matrix_test_test)
# pred.shape
# bin_pred, real = [], np.array(Xy_test_test["class"]).tolist()
# for i in range(len(pred)):
#     print(f'pred = {(round(float(pred[i][0]), 5))},\t real = {real[i]}')
#     bin_pred.append(int(round(float(pred[i][0]), 0))) 
# #np.array(Xy_test["class"]).tolist()
# bin_pred, real
# from sklearn.metrics import classification_report
# print(classification_report(real, bin_pred))